In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
os.environ["KERAS_BACKEND"] = "torch"
import keras
from datamodules import data_processors
import sys
sys.path.append("..")
print(sys.version)
from rusmodules.inverse import inverse_standard

3.13.1 (main, Dec  9 2024, 00:00:00) [GCC 14.2.1 20240912 (Red Hat 14.2.1-3)]


In [2]:
path_modelo = "models/cubico_L1.keras"
modelo = keras.models.load_model(path_modelo)

In [3]:
path_archivo = "../input_data/cubic_1.csv"
full_data = pd.read_csv(path_archivo)
full_data.head()

,phi_a,x_K,eta,beta,phi_K,eig_0,eig_1,eig_2,eig_3,eig_4,...,eig_13,eig_14,eig_15,eig_16,eig_17,eig_18,eig_19,Parallelepiped,Cylinder,Ellipsoid
0,0.843666,0.647473,1.577259,1.574131,0.866533,0.552390,2.341186,2.982731,4.346947,4.543615,...,10.255629,10.934964,12.169362,12.639375,13.023291,17.260278,18.912029,1,0,0
1,0.618911,0.218057,1.216162,1.820191,1.350973,0.451902,1.069440,2.198825,3.912317,3.934114,...,11.230938,11.241764,11.468446,14.561622,14.919805,15.338283,15.666733,1,0,0
2,0.076877,0.245986,1.434582,2.036289,1.322259,0.104398,3.832743,4.758215,5.732612,5.807000,...,15.088024,15.429422,16.241738,16.394792,20.165109,25.500514,26.305482,1,0,0
3,1.459619,0.517388,1.481719,1.376496,1.027000,0.202136,2.142934,2.641222,2.779140,3.088002,...,10.969815,12.756643,13.922951,14.045571,15.047647,18.770442,19.392401,1,0,0
4,0.032508,0.949501,1.839938,0.803203,0.319154,0.002714,10.061990,13.963490,14.997477,17.405756,...,56.063736,70.470168,72.668095,76.780522,93.416045,97.561829,106.012724,1,0,0


In [4]:
N_eig = 19
features = ["eta", "beta"] + list(map(lambda x: "x_" + str(x+1), range(N_eig)))
target = ["phi_a", "phi_K"]

In [5]:
reduced_data = data_processors.preprocess_data(full_data, N_eig, target)
reduced_data.head()

,phi_a,phi_K,eig_0,eta,beta,eig_1,eig_2,eig_3,eig_4,eig_5,...,x_10,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19
0,0.537095,0.551652,0.552390,1.577259,1.574131,0.427134,0.784913,0.686167,0.956716,0.970898,...,0.044254,0.003605,0.062714,0.011232,0.035921,0.065271,0.024853,0.020300,0.224037,0.087339
1,0.394011,0.860056,0.451902,1.216162,1.820191,0.935069,0.486369,0.562026,0.994460,0.833391,...,0.036307,0.021216,0.141992,0.012995,0.000691,0.014469,0.197436,0.022863,0.026711,0.020965
2,0.048942,0.841777,0.104398,1.434582,2.036289,0.260910,0.805500,0.830026,0.987190,0.736554,...,0.024378,0.078955,0.079060,0.006715,0.012978,0.030880,0.005818,0.143328,0.202825,0.030601
3,0.929222,0.653809,0.202136,1.481719,1.376496,0.466650,0.811342,0.950374,0.899980,0.743699,...,0.023561,0.006362,0.055864,0.047249,0.092141,0.060143,0.006323,0.051674,0.191972,0.032072
4,0.020696,0.203180,0.002714,1.839938,0.803203,0.099384,0.720593,0.931056,0.861639,0.698142,...,0.060522,0.001708,0.122370,0.014358,0.135893,0.020733,0.038792,0.156920,0.039106,0.079716


In [6]:
X_data = reduced_data[features]
y_data = reduced_data[target]

In [7]:
X_data.head()

,eta,beta,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,...,x_10,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19
0,1.577259,1.574131,0.070917,0.033923,0.072135,0.010399,0.007201,0.014744,0.043735,0.074120,...,0.044254,0.003605,0.062714,0.011232,0.035921,0.065271,0.024853,0.020300,0.224037,0.087339
1,1.216162,1.820191,0.004432,0.072088,0.109371,0.001391,0.050202,0.004016,0.031852,0.143750,...,0.036307,0.021216,0.141992,0.012995,0.000691,0.014469,0.197436,0.022863,0.026711,0.020965
2,1.434582,2.036289,0.107686,0.035182,0.037042,0.002828,0.078957,0.007227,0.043786,0.025159,...,0.024378,0.078955,0.079060,0.006715,0.012978,0.030880,0.005818,0.143328,0.202825,0.030601
3,1.481719,1.376496,0.058937,0.025695,0.007112,0.015927,0.054878,0.011324,0.097216,0.004204,...,0.023561,0.006362,0.055864,0.047249,0.092141,0.060143,0.006323,0.051674,0.191972,0.032072
4,1.839938,0.803203,0.085480,0.036802,0.009753,0.022717,0.070989,0.010528,0.033579,0.038539,...,0.060522,0.001708,0.122370,0.014358,0.135893,0.020733,0.038792,0.156920,0.039106,0.079716


In [8]:
y_data.head()

,phi_a,phi_K
0,0.537095,0.551652
1,0.394011,0.860056
2,0.048942,0.841777
3,0.929222,0.653809
4,0.020696,0.203180


In [9]:
y_pred = modelo.predict(X_data)

  21/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step  

/home/cubos/.local/lib/python3.13/site-packages/keras/src/backend/torch/numpy.py:114: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:328.)
  return cast(torch.matmul(x1, x2), result_dtype)


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step


In [10]:
print(y_pred)

[[0.53694886 0.5359323 ]
 [0.3710925  0.8610779 ]
 [0.06610139 0.79457784]
 ...
 [0.26572692 0.47376114]
 [0.9689554  0.8753171 ]
 [0.691815   0.85670745]]


In [11]:
X_prueba = X_data.iloc[[1]]
y_prueba = y_data.iloc[1]

In [12]:
X_prueba, y_prueba

(        eta      beta       x_1       x_2       x_3       x_4       x_5  \
 1  1.216162  1.820191  0.004432  0.072088  0.109371  0.001391  0.050202   
 
         x_6       x_7      x_8  ...      x_10      x_11      x_12      x_13  \
 1  0.004016  0.031852  0.14375  ...  0.036307  0.021216  0.141992  0.012995   
 
        x_14      x_15      x_16      x_17      x_18      x_19  
 1  0.000691  0.014469  0.197436  0.022863  0.026711  0.020965  
 
 [1 rows x 21 columns],
 phi_a    0.394011
 phi_K    0.860056
 Name: 1, dtype: float64)

In [13]:
y_pred_prueba = modelo.predict(X_prueba)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


In [14]:
(np.pi/2)*y_pred_prueba

array([[0.58291066, 1.3525778 ]], dtype=float32)

In [15]:
feat_x = list(map(lambda x: "x_" + str(x+1), range(N_eig)))
xs = np.zeros(N_eig+1)
xs[1:] = np.array(X_prueba[feat_x])
xs

array([0.        , 0.00443232, 0.07208808, 0.10937142, 0.00139128,
       0.05020169, 0.00401608, 0.03185233, 0.14375029, 0.02342223,
       0.03630747, 0.02121589, 0.14199204, 0.01299469, 0.00069103,
       0.01446902, 0.19743594, 0.02286263, 0.02671127, 0.02096478])

In [16]:
eta_prueba = float(X_prueba["eta"])
beta_prueba = float(X_prueba["beta"])

/tmp/ipykernel_36132/857982539.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  eta_prueba = float(X_prueba["eta"])
/tmp/ipykernel_36132/857982539.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  beta_prueba = float(X_prueba["beta"])


In [17]:
eta_prueba, beta_prueba

(1.2161615243891672, 1.8201906163928667)

In [18]:
phis = inverse_standard(xs, eta_prueba, beta_prueba, modelo, include_x0=False)
phis

{'eta': 1.2161615243891672, 'beta': 1.8201906163928667, 'x_1': np.float64(0.004432316057840841), 'x_2': np.float64(0.07208807960465148), 'x_3': np.float64(0.1093714226068993), 'x_4': np.float64(0.001391281400654622), 'x_5': np.float64(0.0502016879408379), 'x_6': np.float64(0.004016082810294653), 'x_7': np.float64(0.031852325577391996), 'x_8': np.float64(0.14375028964521294), 'x_9': np.float64(0.02342223012321087), 'x_10': np.float64(0.036307470167929026), 'x_11': np.float64(0.021215890823117842), 'x_12': np.float64(0.1419920437285299), 'x_13': np.float64(0.01299468698362592), 'x_14': np.float64(0.000691031213236402), 'x_15': np.float64(0.014469017206270208), 'x_16': np.float64(0.19743594409746792), 'x_17': np.float64(0.022862630898959756), 'x_18': np.float64(0.026711271734605774), 'x_19': np.float64(0.020964780313600023)}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


array([[0.58291066, 1.3525778 ]], dtype=float32)

In [19]:
a = phis[0,:]/(np.pi/2)
a

array([0.37109244, 0.8610777 ], dtype=float32)

In [20]:
len(a)

2